In [1]:
# basic libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, KFold, cross_val_predict, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.svm import SVR
from sklearn.cluster import KMeans

#others
from xgboost import XGBRegressor
import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker
import time
import xarray as xr
import sherpa
import time

# Variables from config file
from config import BASE_DIR, FILE_NAMES, LABELS, ATTRIBUTES


In [2]:
# Load the datasets
# Load the dataset
# df_metadata = pd.read_excel(f"{BASE_DIR}/FilledDataset2012.xlsx", sheet_name="Header")
# df_data_original = pd.read_csv(f"{BASE_DIR}/dataset.csv")


# load datasets
df_train = pd.read_csv(f"{BASE_DIR}/train.csv")
df_valid = pd.read_csv(f"{BASE_DIR}/valid.csv")
df_test = pd.read_csv(f"{BASE_DIR}/test.csv")

# Nov-Apr = "wet", May-Oct = "dry"
wet = [11, 12, 1, 2, 3, 4]
dry = [5, 6, 7, 8, 9, 10]
df_train['season_dry'] = df_train.apply(lambda row: 1 if row.month in dry else 0, axis=1)
df_train['season_wet'] = df_train.apply(lambda row: 1 if row.month in wet else 0, axis=1)

df_valid['season_dry'] = df_valid.apply(lambda row: 1 if row.month in dry else 0, axis=1)
df_valid['season_wet'] = df_valid.apply(lambda row: 1 if row.month in wet else 0, axis=1)

df_test['season_dry'] = df_test.apply(lambda row: 1 if row.month in dry else 0, axis=1)
df_test['season_wet'] = df_test.apply(lambda row: 1 if row.month in wet else 0, axis=1)

reanalysis_data = [
    'air2m', 'air1000_500', 'hgt500', 'hgt1000', 'omega500',
    'pottemp1000-500', 'pottemp1000-850', 'pr_wtr', 'shum-uwnd-700',
    'shum-uwnd-925', 'shum-vwnd-700', 'shum-vwnd-950', 'shum700', 'shum925', 
    'skt', 'slp'
]

columns = []
for i in range(6):
    for item in reanalysis_data:
        columns.append(f"{item}_{i}")

columns.extend(['data_in', 'lat', 'lon', 'elevation', 'season_wet', 'season_dry'])
for item in columns:
    print(item, end=' ')

air2m_0 air1000_500_0 hgt500_0 hgt1000_0 omega500_0 pottemp1000-500_0 pottemp1000-850_0 pr_wtr_0 shum-uwnd-700_0 shum-uwnd-925_0 shum-vwnd-700_0 shum-vwnd-950_0 shum700_0 shum925_0 skt_0 slp_0 air2m_1 air1000_500_1 hgt500_1 hgt1000_1 omega500_1 pottemp1000-500_1 pottemp1000-850_1 pr_wtr_1 shum-uwnd-700_1 shum-uwnd-925_1 shum-vwnd-700_1 shum-vwnd-950_1 shum700_1 shum925_1 skt_1 slp_1 air2m_2 air1000_500_2 hgt500_2 hgt1000_2 omega500_2 pottemp1000-500_2 pottemp1000-850_2 pr_wtr_2 shum-uwnd-700_2 shum-uwnd-925_2 shum-vwnd-700_2 shum-vwnd-950_2 shum700_2 shum925_2 skt_2 slp_2 air2m_3 air1000_500_3 hgt500_3 hgt1000_3 omega500_3 pottemp1000-500_3 pottemp1000-850_3 pr_wtr_3 shum-uwnd-700_3 shum-uwnd-925_3 shum-vwnd-700_3 shum-vwnd-950_3 shum700_3 shum925_3 skt_3 slp_3 air2m_4 air1000_500_4 hgt500_4 hgt1000_4 omega500_4 pottemp1000-500_4 pottemp1000-850_4 pr_wtr_4 shum-uwnd-700_4 shum-uwnd-925_4 shum-vwnd-700_4 shum-vwnd-950_4 shum700_4 shum925_4 skt_4 slp_4 air2m_5 air1000_500_5 hgt500_5 hgt1

## LOOCV for 100 stations
The following codes runs LOOCV for 100 of randomly selected stations.
Since there are 101 feature variables, any station with less than or equal to 360 training samples are excluded.

In [3]:
station_candidates = []
for name, group in df_train.groupby(by='name'):
    if group.shape[0] > 360:
        station_candidates.append(name)
np.random.seed(42)
stations = np.random.choice(station_candidates, size=100)
stations

array(['PUUKOHOLA HEIAU', 'FIELD 23', 'KAUKONAHUA MAUKA', 'HONOKAA MAUKA',
       'E KAUAI WATER 2', 'Pahoa Beacon', 'FIELD 27 (300)', 'PAIKO DRIVE',
       'HONOMU (1750)', 'POLIPOLI SPRING', 'KAHALUU', 'WAHIAWA MTN',
       'HANA AIRPORT', 'KAPOHO BEACH', '550', 'GAY 10', 'ONOULI (1600)',
       'PUUHINEI 1', 'KAUNALEWA', 'KEMOO CAMP', 'Makaha Country Club',
       'WAIEHU CAMP', 'FIELD 625', 'WAIAHI LOWER', 'MOKUAIKAUA RD',
       'WAIAHI LOWER', 'KUKUI', 'PUU ALALA', 'MAKAHA VALLEY', '549',
       'PUU KAPU RES 3', 'FIELD 4109', 'KAHANA 27', 'LAHAINA',
       'HOLUA CABIN', '542', 'GAGE 29', 'PANILEIHULU', 'KUKUIHAELE LANDG',
       'RESERVOIR 6', 'EKW 1 (120)', 'WAHIAWA MTN', 'KEMOO 3',
       'PANICUM FIELD', 'AIEA FIELD 84', 'FIELD B-8', 'Waiakoali Camp',
       'WHITMORE', 'KUKUIHAELE VILL', 'Makua Valley', 'KAIHULOA',
       'NUUANU RESERV 5', 'FIELD 401', 'FIELD 906 VIL 7', 'WAILUKU RIVER',
       'HALEPIULA 3', 'PUEHU RIDGE', 'KAPAKA', 'WAITA', 'HEADQUARTERS',
       'NEW FI

In [4]:
start = time.time()
results = []
for station in stations:
    df_train_station = df_train[df_train['name'] == station]
    df_test_station  = df_test[df_test['name'] == station]
    if df_train_station.shape[0] == 0 or df_test_station.shape[0] == 0:
        continue
    print("=========================================")
    print(f"Running experiment on {station} station.")
    print(f"There are:")
    print(f"{df_train_station.shape[0]} training data and")
    print(f"{df_test_station.shape[0]} test data")
    print("=========================================")

    # xgboost trains on the entire dataset
    Xtrain = np.array(df_train[columns].drop(labels=["data_in"], axis=1))
    Ytrain = np.array(df_train["data_in"])
    # test on the station data
    Xtest = np.array(df_test_station[columns].drop(labels=["data_in"], axis=1))
    Ytest = np.array(df_test_station["data_in"])

    # hyperparameters obtained by fine tuning
    xgboost = XGBRegressor(
        n_estimators=170,
        learning_rate=0.1,
        max_depth=9,
        verbosity=0
    )

    xgboost.fit(Xtrain, Ytrain)
    print("MSE on xgboost (test) : {:.5f}".format(mean_squared_error(Ytest, xgboost.predict(Xtest))))
    print("MSE on xgboost (train): {:.5f}".format(mean_squared_error(Ytrain, xgboost.predict(Xtrain))))
    mse_xgb = mean_squared_error(Ytest, xgboost.predict(Xtest))

    # linear regression trains on the station dataset
    Xtrain = np.array(df_train_station[columns].drop(labels=["data_in"], axis=1))
    Ytrain = np.array(df_train_station["data_in"])
    # test on the station data
    Xtest = np.array(df_test_station[columns].drop(labels=["data_in"], axis=1))
    Ytest = np.array(df_test_station["data_in"])

    model = LinearRegression()
    model.fit(Xtrain, Ytrain)
    print("MSE on Linear Regression (test) : {:.5f}".format(mean_squared_error(Ytest, model.predict(Xtest))))
    print("MSE on Linear Regression (train): {:.5f}".format(mean_squared_error(Ytrain, model.predict(Xtrain))))
    mse_linear = mean_squared_error(Ytest, model.predict(Xtest))
    
    results.append(
        {"n_samples": df_train_station.shape[0],
         "MSE_xgb": mse_xgb,
         "MSE_linear": mse_linear
        }
    )

sum_weighted_mse_xgb = 0
sum_weighted_mse_linear = 0
total_n_samples = 0
for item in results:
    total_n_samples += item["n_samples"]
    sum_weighted_mse_xgb += item["MSE_xgb"] * item["n_samples"] 
    sum_weighted_mse_linear += item["MSE_linear"] * item["n_samples"] 
    
mean_mse_xgb, mean_mse_linear = sum_weighted_mse_xgb/total_n_samples, sum_weighted_mse_linear/total_n_samples
print("mean of MSE with XGBoost: {:.5f}".format(mean_mse_xgb))
print("mean of MSE with Linear Regression: {:.5f}".format(mean_mse_linear))
    
end = time.time()
print(end - start)

Running experiment on PUUKOHOLA HEIAU station.
There are:
426 training data and
131 test data
MSE on xgboost (test) : 1.26395
MSE on xgboost (train): 4.65344
MSE on Linear Regression (test) : 1.60422
MSE on Linear Regression (train): 1.01516
Running experiment on FIELD 23 station.
There are:
450 training data and
135 test data
MSE on xgboost (test) : 9.01745
MSE on xgboost (train): 4.65344
MSE on Linear Regression (test) : 11.45782
MSE on Linear Regression (train): 12.17854
Running experiment on KAUKONAHUA MAUKA station.
There are:
450 training data and
135 test data
MSE on xgboost (test) : 14.16329
MSE on xgboost (train): 4.65344
MSE on Linear Regression (test) : 16.26509
MSE on Linear Regression (train): 7.72802
Running experiment on HONOKAA MAUKA station.
There are:
449 training data and
135 test data
MSE on xgboost (test) : 29.64510
MSE on xgboost (train): 4.65344
MSE on Linear Regression (test) : 24.83349
MSE on Linear Regression (train): 15.01859
Running experiment on E KAUAI WAT

In [6]:
print("weighted mean of MSE with XGBoost: {:.5f}".format(mean_mse_xgb))
print("weighted mean of MSE with Linear Regression: {:.5f}".format(mean_mse_linear))

weighted mean of MSE with XGBoost: 23.56847
weighted mean of MSE with Linear Regression: 23.40167


## Conclusion
Linear regression models works better for LOOCV for the chosen 100 random stations, when there are as many models as the number of stations. </br>
However, the difference in MSE is within 2%. With such negligible difference, the result should be biased in favor of XGBoost for the following reasons.
<ol>
    <li>Only one model is necessary for XGBoost, whereas we need 2000+ linear regression models.</li>
    <li>XGBoost can make prediction on locations where station does not exist, which would be impossible with linear regression.</li>
    <li>Some stations do not have enough training data for linear regression (at least 100 samples), raising the necessity for hyperparameter tuning for regularization.</li>
</ol>

# Close look at specific stations, HILO, LIHUE, KALAE

In [10]:
station = "HILO"

df_train_station = df_train[df_train['name'] == station]
df_test_station  = df_test[df_test['name'] == station]

# xgboost trains on the entire dataset
Xtrain = np.array(df_train[columns].drop(labels=["data_in"], axis=1))
Ytrain = np.array(df_train["data_in"])
# test on the station data
Xtest = np.array(df_test_station[columns].drop(labels=["data_in"], axis=1))
Ytest = np.array(df_test_station["data_in"])

# hyperparameters obtained by fine tuning
xgboost = XGBRegressor(
    n_estimators=170,
    learning_rate=0.1,
    max_depth=9,
    verbosity=0
)

xgboost.fit(Xtrain, Ytrain)
print("MSE on xgboost (test) : {:.5f}".format(mean_squared_error(Ytest, xgboost.predict(Xtest))))
print("MSE on xgboost (train): {:.5f}".format(mean_squared_error(Ytrain, xgboost.predict(Xtrain))))

# linear regression trains on the station dataset
Xtrain = np.array(df_train_station[columns].drop(labels=["data_in"], axis=1))
Ytrain = np.array(df_train_station["data_in"])
# test on the station data
Xtest = np.array(df_test_station[columns].drop(labels=["data_in"], axis=1))
Ytest = np.array(df_test_station["data_in"])

model = LinearRegression()
model.fit(Xtrain, Ytrain)
print("MSE on Linear Regression (test) : {:.5f}".format(mean_squared_error(Ytest, model.predict(Xtest))))
print("MSE on Linear Regression (train): {:.5f}".format(mean_squared_error(Ytrain, model.predict(Xtrain))))

MSE on xgboost (test) : 36.22984
MSE on xgboost (train): 4.65344
MSE on Linear Regression (test) : 33.09867
MSE on Linear Regression (train): 24.18849


In [11]:
station = "LIHUE"

df_train_station = df_train[df_train['name'] == station]
df_test_station  = df_test[df_test['name'] == station]

# xgboost trains on the entire dataset
Xtrain = np.array(df_train[columns].drop(labels=["data_in"], axis=1))
Ytrain = np.array(df_train["data_in"])
# test on the station data
Xtest = np.array(df_test_station[columns].drop(labels=["data_in"], axis=1))
Ytest = np.array(df_test_station["data_in"])

# hyperparameters obtained by fine tuning
xgboost = XGBRegressor(
    n_estimators=170,
    learning_rate=0.1,
    max_depth=9,
    verbosity=0
)

xgboost.fit(Xtrain, Ytrain)
print("MSE on xgboost (test) : {:.5f}".format(mean_squared_error(Ytest, xgboost.predict(Xtest))))
print("MSE on xgboost (train): {:.5f}".format(mean_squared_error(Ytrain, xgboost.predict(Xtrain))))

# linear regression trains on the station dataset
Xtrain = np.array(df_train_station[columns].drop(labels=["data_in"], axis=1))
Ytrain = np.array(df_train_station["data_in"])
# test on the station data
Xtest = np.array(df_test_station[columns].drop(labels=["data_in"], axis=1))
Ytest = np.array(df_test_station["data_in"])

model = LinearRegression()
model.fit(Xtrain, Ytrain)
print("MSE on Linear Regression (test) : {:.5f}".format(mean_squared_error(Ytest, model.predict(Xtest))))
print("MSE on Linear Regression (train): {:.5f}".format(mean_squared_error(Ytrain, model.predict(Xtrain))))

MSE on xgboost (test) : 5.79774
MSE on xgboost (train): 4.65344
MSE on Linear Regression (test) : 6.01450
MSE on Linear Regression (train): 5.17918


In [13]:
station = "KALAE"

df_train_station = df_train[df_train['name'] == station]
df_test_station  = df_test[df_test['name'] == station]

# xgboost trains on the entire dataset
Xtrain = np.array(df_train[columns].drop(labels=["data_in"], axis=1))
Ytrain = np.array(df_train["data_in"])
# test on the station data
Xtest = np.array(df_test_station[columns].drop(labels=["data_in"], axis=1))
Ytest = np.array(df_test_station["data_in"])

# hyperparameters obtained by fine tuning
xgboost = XGBRegressor(
    n_estimators=170,
    learning_rate=0.1,
    max_depth=9,
    verbosity=0
)

xgboost.fit(Xtrain, Ytrain)
print("MSE on xgboost (test) : {:.5f}".format(mean_squared_error(Ytest, xgboost.predict(Xtest))))
print("MSE on xgboost (train): {:.5f}".format(mean_squared_error(Ytrain, xgboost.predict(Xtrain))))

# linear regression trains on the station dataset
Xtrain = np.array(df_train_station[columns].drop(labels=["data_in"], axis=1))
Ytrain = np.array(df_train_station["data_in"])
# test on the station data
Xtest = np.array(df_test_station[columns].drop(labels=["data_in"], axis=1))
Ytest = np.array(df_test_station["data_in"])

model = LinearRegression()
model.fit(Xtrain, Ytrain)
print("MSE on Linear Regression (test) : {:.5f}".format(mean_squared_error(Ytest, model.predict(Xtest))))
print("MSE on Linear Regression (train): {:.5f}".format(mean_squared_error(Ytrain, model.predict(Xtrain))))

MSE on xgboost (test) : 3.94315
MSE on xgboost (train): 4.65344
MSE on Linear Regression (test) : 5.24540
MSE on Linear Regression (train): 2.00699
